In [ ]:
!pip list | grep torch

torch                            2.4.0+cu121
torchaudio                       2.4.0+cu121
torchsummary                     1.5.1
torchvision                      0.19.0+cu121


In [ ]:
import requests
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
api_key = '8689e34ee2b64fabaff04f67f5f01269'

In [ ]:
# دریافت اخبار
def get_news(api_key):
    url = f'https://newsapi.org/v2/everything?q=cryptocurrency&apiKey={api_key}'
    response = requests.get(url)
    return response.json()

In [ ]:
get_news('8689e34ee2b64fabaff04f67f5f01269')['articles'][0]

{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': 'Gaby Del Valle',
 'title': 'Trump is launching a cryptocurrency platform, and we have no idea what it does',
 'description': 'There are few public details about the platform, which Trump announced in a post on Truth Social, but Trump’s sons have suggested it will help unbanked people.',
 'url': 'https://www.theverge.com/2024/8/22/24226184/trump-defiant-ones-cryptocurrency-platform',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/_Nmh513ei1pcO8JPH6S8zPZY9Xk=/0x0:3000x2000/1200x628/filters:focal(1500x1000:1501x1001)/cdn.vox-cdn.com/uploads/chorus_asset/file/25531774/STK175_Trump_Fist_Glitch.jpg',
 'publishedAt': '2024-08-22T18:51:46Z',
 'content': 'Trump is launching a cryptocurrency platform, and we have no idea what it does\r\nTrump is launching a cryptocurrency platform, and we have no idea what it does\r\n / Trump and his sons have suggested th… [+3560 chars]'}

In [ ]:
# دریافت قیمت‌ های ارز دیجیتال
def get_crypto_prices():
    url = 'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'sparkline': False
    }
    response = requests.get(url, params=params)
    return response.json()

In [ ]:
get_crypto_prices()[0]

{'id': 'bitcoin',
 'symbol': 'btc',
 'name': 'Bitcoin',
 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
 'current_price': 60450,
 'market_cap': 1195343466776,
 'market_cap_rank': 1,
 'fully_diluted_valuation': 1271203890690,
 'total_volume': 32737649542,
 'high_24h': 61150,
 'low_24h': 58712,
 'price_change_24h': 1737.66,
 'price_change_percentage_24h': 2.95963,
 'market_cap_change_24h': 36038094890,
 'market_cap_change_percentage_24h': 3.10859,
 'circulating_supply': 19746803.0,
 'total_supply': 21000000.0,
 'max_supply': 21000000.0,
 'ath': 73738,
 'ath_change_percentage': -17.90483,
 'ath_date': '2024-03-14T07:10:36.635Z',
 'atl': 67.81,
 'atl_change_percentage': 89173.22919,
 'atl_date': '2013-07-06T00:00:00.000Z',
 'roi': None,
 'last_updated': '2024-08-29T18:04:52.333Z'}

In [ ]:
# تحلیل داده ها با تعیین مدل و دستگاه
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert/distilbert-base-uncased-finetuned-sst-2-english', device=0)

In [ ]:
def analyze_sentiments(news_articles):
    titles_and_descriptions = [
        (article['title'] if article['title'] else "") + ". " +
        (article['description'] if article['description'] else "")
        for article in news_articles
    ]
    sentiments = sentiment_analyzer(titles_and_descriptions)
    return [sent['label'] for sent in sentiments]

In [ ]:
# ترکیب داده‌ها
def combine_data(prices, sentiments):
    df = pd.DataFrame(prices)
    df['sentiment'] = sentiments
    return df

In [ ]:
# پیش‌بینی قیمت‌ها
def predict_prices(df):
    model = LinearRegression()
    X = df[['sentiment']].applymap(lambda x: 1 if x == 'POSITIVE' else 0)
    y = df['current_price']
    model.fit(X, y)
    predictions = model.predict(X)
    return predictions

In [ ]:
# بارگذاری مدل و توکنایزر GPT-3
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# تابع تولید متن
def generate_text(predictions):
    input_text = f"Based on the latest predictions, the price of cryptocurrencies is expected to "
    for i, prediction in enumerate(predictions):
        input_text += f"coin {i+1}: ${prediction:.2f}, "
    input_text = input_text.rstrip(", ") + "."

    # تولید متن با استفاده از مدل GPT-3
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=2000, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
news = get_news(api_key)
prices = get_crypto_prices()
sentiments = analyze_sentiments(news['articles'])
df = combine_data(prices, sentiments)
predictions = predict_prices(df)
generated_text = generate_text(predictions)
# return jsonify({'predictions': predictions.tolist(), 'generated_text': generated_text})

<ipython-input-17-f784a6090db7>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = df[['sentiment']].applymap(lambda x: 1 if x == 'POSITIVE' else 0)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
predictions.tolist()

[1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 2532.2981549056276,
 2532.2981549056276,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 2532.2981549056276,
 2532.2981549056276,
 1941.5782949674685,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 2532.2981549056276,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949674685,
 1941.5782949

In [ ]:
generated_text

"Based on the latest predictions, the price of cryptocurrencies is expected to coin 1: $1941.58, coin 2: $2532.30, coin 3: $1941.58, coin 4: $2532.30, coin 5: $1941.58, coin 6: $1941.58, coin 7: $1941.58, coin 8: $2532.30, coin 9: $1941.58, coin 10: $2532.30, coin 11: $1941.58, coin 12: $2532.30, coin 13: $2532.30, coin 14: $1941.58, coin 15: $2532.30, coin 16: $1941.58, coin 17: $2532.30, coin 18: $1941.58, coin 19: $1941.58, coin 20: $1941.58, coin 21: $1941.58, coin 22: $2532.30, coin 23: $2532.30, coin 24: $1941.58, coin 25: $1941.58, coin 26: $2532.30, coin 27: $1941.58, coin 28: $1941.58, coin 29: $1941.58, coin 30: $1941.58, coin 31: $1941.58, coin 32: $1941.58, coin 33: $1941.58, coin 34: $1941.58, coin 35: $1941.58, coin 36: $1941.58, coin 37: $2532.30, coin 38: $1941.58, coin 39: $1941.58, coin 40: $1941.58, coin 41: $1941.58, coin 42: $1941.58, coin 43: $1941.58, coin 44: $1941.58, coin 45: $1941.58, coin 46: $1941.58, coin 47: $1941.58, coin 48: $1941.58, coin 49: $1941.58,